# Библиотеки

In [32]:
import pandas as pd
import numpy as np
import neptune

from NN import simple_torchpl
from load_data import load
from pl_framework import nn_training
from cv import get_indices

import torch

In [33]:
neptune.init('iliaavilov/SIBUR')

Project(iliaavilov/SIBUR)

In [34]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzI0ZTMzMDQtZTFmZi00ZjgxLWI4NGQtZGJiOWQyNDJiYjE5In0="
os.environ['NEPTUNE_PROJECT']="iliaavilov/SIBUR"
os.environ['NEPTUNE_NOTEBOOK_ID']="6eddcb21-b0fa-4073-a5c6-f26572800a8b"
os.environ['NEPTUNE_NOTEBOOK_PATH']="Desktop/Projects/sibur_2020/modeling.ipynb"

# Константы

In [35]:
data_path = 'data/'
random_state = 54321
pd.options.display.max_rows = 999
features = ['A_rate', 'A_CH4', 'A_C2H6', 'A_C3H8', 'A_iC4H10', 'A_nC4H10',
            'A_iC5H12', 'A_nC5H12', 'A_C6H14', 'B_rate']

# Загрузка данных

In [36]:
train_features, train_targets, test_features = load(data_path)

In [37]:
train_features = train_features[(train_features['timestamp'] > pd.to_datetime('2020-02-15 00:00:00'))].reset_index(drop = True)
train_targets = train_targets[(train_targets['timestamp'] > pd.to_datetime('2020-02-15 00:00:00'))].reset_index(drop = True)

In [38]:
cv = get_indices(train_targets, [(pd.to_datetime('2020-03-01 00:00:00'), pd.to_datetime('2020-03-15 00:00:00')),
                                 (pd.to_datetime('2020-03-15 00:00:00'), pd.to_datetime('2020-03-31 00:00:00')),
#                                 (pd.to_datetime('2020-03-31 00:00:00'), pd.to_datetime('2020-04-15 00:00:00')),
                                 (pd.to_datetime('2020-04-15 00:00:00'), pd.to_datetime('2020-04-30 00:00:00'))
                                ]
                )

In [39]:
train_targets = train_targets.drop('timestamp', axis = 'columns')
test_features = test_features.drop('timestamp', axis = 'columns')
train_features = train_features.drop('timestamp', axis = 'columns')

# Предсказания для теста

In [40]:
params_model = {'seq_len': 5, 'n_h_1': 1051, 'n_h_2': 1584, 'batch_size': 2165, 'p_1': 0.19035125140405518, 'p_2': 0.22618762429059036, 'activation1': 'Hardshrink', 'activation2': 'ELU', 'optimizer': 'Adadelta', 'lr': 0.00033405740435190565, 'weight_decay': 0.8727244041780381, 'n_out': 4, 'mean_best_iter': 1116, 'n_in': 10}

In [41]:
def make_seqs(n, data):
    final_X = []

    def seq2seq(x):
        seqs_X.append(np.array(x).reshape(-1, 1))
        return (5)

    for col in data.columns:
        seqs_X = []
        data[col].rolling(n).apply(seq2seq)
        seqs_X = np.array(seqs_X)
        final_X.append(seqs_X)

    final_X = np.concatenate(final_X, axis=2)
    return (final_X)


all_features = train_features.append(test_features, ignore_index = True)

seq_len = params_model['seq_len']
params_model.pop('seq_len')
batch_size = params_model['batch_size']
params_model.pop('batch_size')
mean_best_iter = params_model['mean_best_iter']
params_model.pop('mean_best_iter')

all_features = make_seqs(seq_len, all_features)
X_train = all_features[:(train_features.shape[0]-seq_len+1)]
X_test = all_features[(train_features.shape[0]+1):]

X_train = X_train
X_test = X_test

y_train  = train_targets
y_train = y_train[(seq_len-1):].reset_index(drop = True)

In [42]:
t = 0
for fold in cv:
    fold[0] = fold[0][(seq_len-1):]
    for idx1 in range(0, len(fold[0])):
        fold[0][idx1] = fold[0][idx1] - seq_len + 1

    for idx2 in range(0, len(fold[1])):
        fold[1][idx2] = fold[1][idx2] - seq_len + 1
    cv[t] = fold
    t = t + 1

In [44]:
cv_new = [cv[0]]

In [47]:
my_boiii = nn_training(simple_torchpl, X_train, y_train)

my_boiii.train(min_epochs=mean_best_iter,
               max_epochs=mean_best_iter,
               model_params=params_model,
               batch_size=batch_size,
               fold=cv_new[0],
               val_fold=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | activ1       | Hardshrink | 0     
1 | activ2       | ELU        | 0     
2 | lstm         | LSTM       | 4 M   
3 | dropout1     | Dropout    | 0     
4 | dropout2     | Dropout    | 0     
5 | linear1      | Linear     | 1 M   
6 | linear_final | Linear     | 6 K   
7 | loss         | MSELoss    | 0     
8 | loss_sec     | MAPE       | 0     


train_loss: 1.0024267435073853
val_loss: 1.0017380714416504
train_loss: 1.0024231672286987
val_loss: 1.0016412734985352
train_loss: 1.0023366212844849
val_loss: 1.0015569925308228
train_loss: 1.0023229122161865
val_loss: 1.001462697982788
train_loss: 1.002064824104309
val_loss: 1.001360297203064
train_loss: 1.0019662380218506
val_loss: 1.0012626647949219
train_loss: 1.0018279552459717
val_loss: 1.0011723041534424
train_loss: 1.0018417835235596
val_loss: 1.0010777711868286
train_loss: 1.001786708831787
val_loss: 1.0009840726852417
train_loss: 1.0017608404159546
val_loss: 1.00089430809021
train_loss: 1.0017294883728027
val_loss: 1.0007975101470947
train_loss: 1.0014296770095825
val_loss: 1.0006991624832153
train_loss: 1.0012725591659546
val_loss: 1.0006036758422852
train_loss: 1.001070261001587
val_loss: 1.0005037784576416
train_loss: 1.001404881477356
val_loss: 1.0003929138183594
train_loss: 1.0012776851654053
val_loss: 1.0002919435501099
train_loss: 1.0009126663208008
val_loss: 1.00019

train_loss: 0.9818888902664185
val_loss: 0.9814982414245605
train_loss: 0.9816872477531433
val_loss: 0.9813014268875122
train_loss: 0.9818810224533081
val_loss: 0.9811009764671326
train_loss: 0.9817635416984558
val_loss: 0.9809029698371887
train_loss: 0.9814500212669373
val_loss: 0.9806981682777405
train_loss: 0.9811484813690186
val_loss: 0.9804994463920593
train_loss: 0.9805392622947693
val_loss: 0.9803034663200378
train_loss: 0.9808362126350403
val_loss: 0.9801037907600403
train_loss: 0.9808019995689392
val_loss: 0.9799028038978577
train_loss: 0.9806643128395081
val_loss: 0.979705274105072
train_loss: 0.9803046584129333
val_loss: 0.9795090556144714
train_loss: 0.9797331690788269
val_loss: 0.9793137907981873
train_loss: 0.979727029800415
val_loss: 0.9791136384010315
train_loss: 0.9797105193138123
val_loss: 0.9789136052131653
train_loss: 0.9797142148017883
val_loss: 0.9787169098854065
train_loss: 0.9790221452713013
val_loss: 0.9785206317901611
train_loss: 0.9792563915252686
val_loss: 0

C:\Users\Sibmice\anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [ ]:
my_model1 = my_boiii.model
my_model1.eval()

In [ ]:
predictions = my_model1(torch.from_numpy(X_test.values).float()).detach().numpy()
submission = pd.DataFrame(predictions, columns = ['B_C2H6', 'B_C3H8', 'B_iC4H10', 'B_nC4H10'])
submission['timestamp'] = test_features['timestamp'].values
submission.to_csv('submission.csv', index = False)

In [ ]:
#neptune.project.get_experiments('SIB-71')[0].log_metric('leaderboard_mape', 7.4781)

In [ ]:
X_train

In [ ]:
def make_seqs(n, data):
    final_X = []

    def seq2seq(x):
        seqs_X.append(np.array(x).reshape(-1, 1))
        return (5)

    for col in data.columns:
        seqs_X = []
        data[col].rolling(n).apply(seq2seq)
        seqs_X = np.array(seqs_X)
        final_X.append(seqs_X)

    final_X = np.concatenate(final_X, axis=2)
    return (final_X)

In [ ]:
final_X = make_seqs(2, X_train)

In [ ]:
final_X[0][-1]

In [ ]:
tt = [10, 20, 40]

tt-([5]*len(tt))

In [ ]:
final_X[0]